# Tension Terminator Model Trainer

## Set global variables

In [1]:
# Set Variables
BASE_MODEL = "yolov8s"
TRAIN_EPOCHS = 100
TRAIN_SIZE = 320

TT_PROJECTNAME = "TensionTerminator"

## Mount Google Drive to backup files

In [ ]:
# Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

google_drive_target_folder = '/content/drive/My Drive/'+TT_PROJECTNAME+'/'
# Check if the target folder exists in Google Drive, create if it doesn't
print("\n\info")
if not os.path.exists(google_drive_target_folder):
    os.makedirs(google_drive_target_folder)
    print(f"Created folder: {google_drive_target_folder}")
print("Google Drive Target Folder: " + google_drive_target_folder)

# Install Dependencies

In [ ]:
pip install ultralytics

In [ ]:
!pip install roboflow

## Python imports

In [ ]:
import os
import random
import yaml
import shutil
from ultralytics import YOLO
from roboflow import Roboflow
from google.colab import userdata, files
from datetime import datetime

## Loading dataset to train on

In [ ]:
secret_key = userdata.get('rb-key')
rf = Roboflow(api_key=secret_key)
project = rf.workspace("juzaworkspace").project("overheadpersondetection")
dataset = project.version(3).download("yolov8")
DATASET_DIR = dataset.location
DATA_YAML = DATASET_DIR + "/data.yaml"
print("\n\info")
print("Data saved to: " + DATASET_DIR)
print("Data YAML: " + DATA_YAML)

There is a bug in the data.yaml of robfolow yolov7. This script corrects it.

In [ ]:
# corrects the folders in the data.yaml

# Path to the YAML file
yaml_file_path = DATASET_DIR + 'data.yaml'

# Read the YAML file
with open(DATA_YAML) as file:
    data = yaml.safe_load(file)

# Update the paths
data['train'] = DATASET_DIR + '/train/images'
data['val'] = DATASET_DIR + '/valid/images'
data['test'] = DATASET_DIR + '/test/images'

# Write the updated data back to the YAML file
with open(DATA_YAML, 'w') as file:
    yaml.dump(data, file)

print("Updated data.yaml successfully.")


## Train a new model

In [ ]:
# Variables already set, but u can run to re-train just this cell and modify the vars
#TRAIN_MODEL = "yolov8s"
#TRAIN_EPOCHS = 1
#TRAIN_SIZE = 320

# Load a model
modelpt = BASE_MODEL + ".pt"
model = YOLO(modelpt)  # load a pretrained model (recommended for training)

# train the model
model.train(data=DATA_YAML, epochs=TRAIN_EPOCHS, imgsz=TRAIN_SIZE)

## Save the model to Google Drive

In [ ]:
# Define the base directory where the runs are stored
base_dir = '/content/runs/detect/'

# Find the latest run directory
latest_run_dir = max([os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))], key=os.path.getmtime)

# Construct the path to the best weights file
TRAINED_WEIGHTS = os.path.join(latest_run_dir, 'weights/best.pt')
print(TRAINED_WEIGHTS)

In [ ]:
# Define the base directory where the runs are stored
base_dir = '/content/runs/detect/'

# Find the latest run directory
latest_run_dir = max([os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))], key=os.path.getmtime)

# Construct the path to the best weights file
TRAINED_WEIGHTS = os.path.join(latest_run_dir, 'weights/best.pt')
print("Original trained weights path: ", TRAINED_WEIGHTS)

# Get the current timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Construct the new file name with timestamp
new_file_name = f"TTmodel_{BASE_MODEL}_{timestamp}.pt"
new_file_path = os.path.join(latest_run_dir, 'weights', new_file_name)

# Copy the file with timestamp in the same directory
shutil.copy2(TRAINED_WEIGHTS, new_file_path)
print("New file path with timestamp: ", new_file_path)

# Make sure the target folder exists
if not os.path.exists(google_drive_target_folder):
    os.makedirs(google_drive_target_folder)

# Construct the target file path in Google Drive
google_drive_target_path = os.path.join(google_drive_target_folder, new_file_name)

# Copy the file to Google Drive
shutil.copy2(new_file_path, google_drive_target_path)
print("File copied to Google Drive: ", google_drive_target_path)


## Evaluate Performance

In [ ]:
# Evaluate the model's performance on the validation set
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

Testing on 3 random files

In [ ]:
# Define the path to the test images directory
test_images_dir = DATASET_DIR + "/test/images"

# Get a list of all files in the test images directory
all_test_images = os.listdir(test_images_dir)

# Randomly select 3 images
random_test_images = random.sample(all_test_images, 3)

# Predict on each of the 3 random images
for image_name in random_test_images:
    image_path = os.path.join(test_images_dir, image_name)
    results = model(image_path)  # Perform prediction
    print(f"Results for {image_name}:")
    print(results)  # or use results.show() or results.save() based on your requirement

    # Optional: display the image with predictions
    # results.show()


In [ ]:
#metrics = model.val()  # evaluate model performance on the validation set
#results = model("/content/OverheadPersonDetection-3/test/images/test_000010_0774_right_png.rf.928e885b5917efafe8edefaeb37e2500.jpg")  # predict on an image
#path = model.export(format="onnx")  # export the model to ONNX format

In [ ]:
#from ultralytics.utils.benchmarks import benchmark

# Benchmark on GPU
#benchmark(model=TRAINED_WEIGHTS, data=TRAIN_DATA, imgsz=TRAIN_SIZE, half=False, device=0)

# END